In [ ]:
import os
import numpy as np
from ipywidgets import interact
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, losses, optimizers, models

from loss import *
from network import *

In [ ]:
def inter_plot(img1, img2):
    max_idx = len(img1)
    vmin = np.maximum(img1.max(axis = (1, 2, 3)), img2[-1].max(axis = (1, 2, 3)))
    vmax = np.minimum(img1.min(axis = (1, 2, 3)), img2[-1].min(axis = (1, 2, 3)))
    def plot(idx=0):
        plt.figure(figsize=(12, 6))
        
        plt.subplot(121)
        plt.imshow(img1[idx, ..., 0], vmin=0, vmax=1, cmap='gray')
        plt.title("Input")
        
        plt.subplot(122)
        plt.imshow(img2[-1][idx, ..., 0], vmin=0, vmax=1, cmap='gray')
        plt.title("Recon")
        
    interact(plot, idx=(0, max_idx, 1))

In [ ]:
t1 = np.load('./data/t1.npy')
t2 = np.load('./data/t2.npy')

t1 = t1/t1.max(axis=(1, 2, 3), keepdims=True)
t2 = t2/t2.max(axis=(1, 2, 3), keepdims=True)

In [ ]:
model = VAE(latent_dim=128, last_activation='sigmoid')

recon_loss = losses.mse(model.input, model.output[-1])
kl_loss = 1 + model.output[1] - tf.square(model.output[0]) - tf.exp(model.output[1])
kl_loss = tf.reduce_sum(kl_loss, axis=-1)
loss = 100*tf.reduce_mean(recon_loss) - tf.reduce_mean(kl_loss)
model.add_loss(loss)

model.compile(optimizer=optimizers.Adam(0.0001), loss = None)

In [ ]:
model.fit(t1[:318], epochs=50, batch_size = 8, validation_data=[t1[318:], None, None])

In [ ]:
pred = model.predict(t1[:24])

In [ ]:
inter_plot(t1[:24], pred)